## Predictions for the next 50 years (2019 - 2069)
Includes exogenous locations outside of North Carolina to improve model predictions

In [38]:
import os
import getpass
import pandas as pd
import numpy as np
import csv
import statsmodels.api as sm
import warnings
import math
import copy
import multiprocessing
import traceback
import hashlib
import signal
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from itertools import combinations
from scipy import stats
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from datetime import datetime
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")

try: 
    __file__
except:
    curr_dir = os.path.abspath('')
else:
    curr_dir = os.path.dirname(os.path.abspath(__file__))
    
app_root = curr_dir if os.path.basename(curr_dir) != "src" else os.path.dirname(curr_dir)

if getpass.getuser() == "rainfalld":  # systemd daemon
    home = os.path.expanduser("~")
    destdir = home                    # /var/cache/rainfall-predictor
else:
    destdir = os.path.join(app_root,'data','manipulated_data')      # if dev, stay in repository


file = os.path.join(destdir,'rainfalldata.csv')
rd = pd.read_csv(file)
file2 = os.path.join(destdir,'ncrainfalldata.csv')
ncrd = pd.read_csv(file2)
rd.Date = pd.to_datetime(rd.Date)
rd = rd.set_index('Date')
ncrd.Date = pd.to_datetime(ncrd.Date)
ncrd = ncrd.set_index('Date')

In [2]:
import json
# this cell takes the stored exogen dictionary that is stored in the Data_Wrangling_CAP1 jupyter notebook
# that was imported above.
try:
    raise NameError()
    %store -r exogen
except NameError:
    f = open(os.path.join(destdir,"exogen.json"),"r")
    exogen = json.load(f)      # read from file, passed from Data_Wrangling
    f.close()


### Function Library
Handles ...

In [ ]:
def sarima_model_creation(data, p, d, q, P, D, Q, m, exog=None):
    my_order = [p,d,q]
    my_sorder = [P,D,Q,m]
    sarimamod = sm.tsa.statespace.SARIMAX(data, exog, order=my_order, seasonal_order=my_sorder, 
                                          enforce_stationarity=False, enforce_invertibility=False,
                                          initialization='approximate_diffuse')
    model_fit = sarimamod.fit(disp=0)   # start_params=[0, 0, 0, 0, 1])
    return(model_fit)

In [ ]:
def model_creation_pred_one_step(train_data, test_data, exotrain=None, exotest=None, progress_bar=None):
    ''' recursively makes forecast based on provided data for the next month
        args: train_data = large data set to base predictions on
              test_data  = decreasing dataset of data to test model
              exotrain   = exogenous location data that matches the same timeframe of train_data but was not included
              exotest    = exogenous location data that matches the same timeframe of test_data but was not included
        returns: A list of all predictions for the location matching the entire test_data timeframe
    '''
    list_one_step = []
    
    nextMonth = model_based_forecast(train_data, exotrain)
    list_one_step.append(nextMonth[0])             # captures prediction
    progress_bar.update()

    # if test data exists
    if len(test_data) > 1:
        # increment data for next month's iteration
        train_data = pd.concat([train_data, test_data.iloc[[0]]])
        test_data = test_data.drop(test_data.index[0], axis = 0)
        if exotrain is not None:
            exotrain = pd.concat([exotrain, exotest.iloc[[0]]])
            exotest = exotest.drop(exotest.index[0], axis = 0)

        # execute & capture future predictions
        futurePredictions = model_creation_pred_one_step(train_data, test_data, exotrain, exotest, progress_bar)
        # add to list
        list_one_step.extend(futurePredictions)
        
    return(list_one_step)

def model_based_forecast(train_data, exotrain=None):
    ''' creates model from training data & makes a forecast
        args: train_data = DataFrame to build forecasting model
              exotrain   = DataFrame of exogenous location's rainfall data
        returns: FLOAT value of next month's forecast value
    '''
    mod = sarima_model_creation(train_data, p=4, d=0, q=3, P=3, D=0, Q=4, m=12, exog=exotrain)
    # if exists, passing exotrain's prevMonth (december, for forecasting jan), otherwise only forcast based on model
    nextMonth = mod.forecast() if exotrain is None else mod.forecast(exog=exotrain.iloc[[-1]])       # turnary assignment expression
    return(nextMonth)



In [ ]:
def update_JSON_file(filename, adjustfn, arglist=(), kwargs={}, sort=True):
    ''' Generic function to handle JSON file updates.  Reads-in entire file, 
        federates out updates with adjustment fn's, and then overwrites original file completely
        Handles FileNotFoundError & JSONDecodeError automatically.
        args: filename = json-encoded file on disk
              adjustfn = function to perform adjustments to loaded dictionary file
              arglist = positional args to pass on to adjustfn
              kwargs = keyword args to pass on to adjustfn
              sort = flag to auto-sort keys when saving to file [Default = True]
        returns: dictionary object that was updated and saved to file
    '''
    def default_dict_adjustfn(data, key, value):
        ''' Generic default function for updating a basic dictionary data file (top level keys only)
            args: data = dictionary representation of JSON data from file
                  key = key name to enter into dictionary
                  value = value to enter into dictionary[key]
            returns: Updated dictionary with key/value added
        '''
        data[key] = value
        return(data)
    
    def default_list_adjustfn(data, value):
        ''' Generic default function for updating a basic list data file (add to bottom of list)
            args: data = list representation of JSON data from file
                  value = value to append to end of list, list[len(list)] = value
            returns: Updated list with value appended
        '''
        data.append(value)
        return(data)
    
    loaded = False
    while not loaded:
        try:
            file = open(filename, "r+")
            json_data = json.loads(file.read())
        except FileNotFoundError:
            open(filename, "w+").close()       # create file on disk
            continue
        except json.JSONDecodeError:
            json_data = {}
        
        loaded = True
        if adjustfn is not None:
            json_data = adjustfn(json_data, *arglist, **kwargs)
        else:
            if isinstance(json_data, dict):
                json_data = default_dict_adjustfn(json_data, *arglist, **kwargs)
            elif isinstance(json_data, list):
                json_data = default_list_adjustfn(json_data, *arglist, **kwargs)
            else:
                raise ValueError('Unable to adjust JSON since function not provided or file not of type dict or list!')
        
        file.seek(0)                           # Go to first line, first column of file
        file.write( json.dumps(json_data, sort_keys=sort, indent=4)+'\n')
        file.truncate()                        # end file here, delete anything after the current file position
        file.close()
    
    return(json_data)

### Data Evaluation


In [ ]:
bettermae_results_filename = os.path.join(destdir,"allBetterMAE.json")

files=[]
while (len(files) > 0):                          # reset results on new run
    try:
        os.remove( files[-1] )
    except FileNotFoundError:                    # ignore since non-exist is the desired state
        pass
    except OSError as err:
        traceback.print_exception(type(err), err, err.__traceback__)
    finally:
        files.pop()


data_test_percentage = 0.2                                           # 20%
num_single_predictions = math.ceil(rd.shape[0]*data_test_percentage)
num_all_predictions = len(l_o_dfs.items())*num_single_predictions    # keymae predictions amount
# num_all_exmae = 0
# for key,value in l_o_dfs.items():
#     num_all_exmae += len(value)                                      # exmae predictions amount
# num_all_predictions += num_all_exmae*num_single_predictions

# keymae_storage = {}    
# # tqdmformat = '{desc}: |{bar}|{percentage:3.0f}%'
# total_progress = tqdm(desc="Full Calculation:", total=num_all_predictions, position=0)
# keymae_progress = tqdm(desc="Finding keymaes:", total=len(l_o_dfs.items()), position=1)
# exmae_progress = tqdm(desc="Evaluating exmaes:", total=num_all_exmae, position=2)

# try:
#     # Solve for targetloc Keymae Values first
#     print("============== KEYMAE EVALUATION ===============")
#     pbars = { 'total_pbar': total_progress, 'keymae_pbar': keymae_progress, 'exmae_pbar': exmae_progress }
#     keymae_storage = targetloc_vars(rd, list(l_o_dfs.keys()), data_test_percentage, pbars)

#     # Solve for exmae values of each combination of targetloc and matching exogenous variable
#     print("\n=============== EXMAE EVALUATION ===============")
#     for key,value in l_o_dfs.items():
#         print("\nFinding exmaes values for {0}:".format(key))
#         targetloc_obj = { 'data': rd[key], 'name': key, 'keymae': keymae_storage[key] }
#         exogenous_var(targetloc_obj, data_test_percentage, value, pbars)

# except KeyboardInterrupt:
#     print("MANUAL EXIT: Program interrupted by user.", flush=True)
#     raise SystemExit(2)
# except Exception as err:
#     print("ERROR: {}".format(err), flush=True)
#     traceback.print_exception(type(err), err, err.__traceback__)
#     raise SystemExit(1)
# else:
#     print("\n==== EXOGENOUS VARIABLE EVALUATION COMPLETE ====\n")
# finally:
#     keymae_progress.close()
#     exmae_progress.close()
#     total_progress.close()


In [ ]:
# Load any found exogenous locations that improve model accuracy for certain cities
try:
    with open(bettermae_results_filename, "r") as f:
        improvement_exog = json.loads(f.read())
except FileNotFoundError as err:
    print("File ({0}) cannot be found. Exiting...".format(bettermae_results_filename))
    SystemExit(1)
except json.JSONDecodeError as err:
    print("File ({0}) has been corrupted. Cannot read json.".format(bettermae_results_filename))
    print("Exiting...")
    SystemExit(1)
else:
    with_exogs = list(improvement_exog.keys())    # Extract target locations with exogenous locations that improve the target predictions
    ncrd2 = ncrd.copy()
    ncrd_less = ncrd2.drop(with_exogs,axis=1)     # Separate out locations with exogenous locations from other data
    print("with_exogs = {0}".format(with_exogs))

In [ ]:
# Dependency Sort
# if a city's prediction model is improved by an exogenous location than it is dependent on the exogenous location's predictions
# Cities without exogenous locations are least dependent so they will be solved first

In [ ]:
def prediction_fx(data, begin, end):
    base = datetime.strptime(begin,'%Y-%m-%d')
    date_list = [base + relativedelta(months=x) for x in range(600)]
    prediction1_df = pd.DataFrame(index=date_list)
    for col in tqdm(data.columns):
        loc = data[col]
        mod_fit1 = sarima_model_creation(loc, 4,0,3,3,0,4,12)
        point_predictions = pd.DataFrame(mod_fit1.predict(start=begin, end=end), columns=[col])
        future_pred1 = mod_fit1.get_prediction(start=begin, end=end)
        future_pred1_ci = future_pred1.conf_int(alpha=0.5)
        point_predictions_df = pd.merge(point_predictions, future_pred1_ci, left_index=True, right_index=True)
        prediction1_df = pd.merge(prediction1_df, point_predictions_df, left_index=True, right_index=True)
    return(prediction1_df)

In [ ]:
def prediction_exog_fx2(data, exog_dict, begin, end):
    base = datetime.strptime(begin,'%Y-%m-%d')
    end_date = datetime.strptime(begin,'%Y-%m-%d')
    num_predicted_months = (base.year - end_date.year) * 12 + base.month - end_date.month
    date_list = [base + relativedelta(months=x) for x in range(num_predicted_months)]
    prediction_df = pd.DataFrame(index = date_list)
    pred_val_df = pd.DataFrame(index = date_list)
    exog_predictions_df = pd.DataFrame(index = date_list)
    for key,value in tqdm(exog_dict.items()):
        loc = data[key]
        mod_fit1 = sarima_model_creation(loc, 4,0,3,3,0,4, 12,exog=value)
        if value.shape[1] > 1:
            shap = value.shape[1]
            for i in range(shap):
                exog_mod_fit = sarima_model_creation(value.iloc[:,i],4,0,3,3,0,4,12)
                e_preds2 = pd.DataFrame(exog_mod_fit.predict(start=begin, end=end))
                if i is 0:
                    exog_predictions_df = e_preds2
                else:
                    exog_predictions_df = pd.merge(exog_predictions_df, e_preds2, left_index=True, 
                                                   right_index=True)
        else:
            exog_mod_fit = sarima_model_creation(value, 4,0,3,3,0,4,12)
            exog_predictions_df = pd.DataFrame(exog_mod_fit.predict(start=begin, end=end))
        future_pred = mod_fit1.get_prediction(exog=exog_predictions_df,start=begin, end=end)
        future_pred_ci = future_pred.conf_int(alpha=0.5)
        future_pred_val= pd.DataFrame(mod_fit1.predict(exog=exog_predictions_df, start=begin, end=end), 
                                      columns = [key])
        future_pred_full = pd.merge(future_pred_val, future_pred_ci, left_index=True, right_index=True)
        prediction_df = pd.merge(prediction_df, future_pred_full, left_index=True, right_index=True)
    return(prediction_df)

In [ ]:
# Build dictionary of dataframes containing exogenous location rainfall data organized by target location
exo_var_dict2 = {}
for i in range(0,len(with_exogs)):
    targetloc = with_exogs[i]
    bestmae_combo = None
    for combo,mae in improvement_exog[targetloc]['exogen'].items():
        if bestmae_combo is None:                         # BubbleSort always starts arbitrary
            bestmae_combo = combo
        elif mae < improvement_exog[targetloc]['exogen'][bestmae_combo]:    # Found better MAE
            bestmae_combo = combo
        else:
            pass
    # with best combo, split if it uses multiple exog_vars
    combo_exogloc = bestmae_combo.split('|')
    exog_raindata = rd[[combo_exogloc[0]]]
    for e in range(1,len(combo_exogloc)):
        exog_raindata = pd.merge(exog_raindata, rd[[combo_exogloc[e]]], left_index=True, right_index=True)
    exo_var_dict2[targetloc] = exog_raindata

print(exo_var_dict2)

In [47]:
YEARS_IN_FUTURE = 50

last_date_in_data = (rd.iloc[[-1]]).index[0].to_pydatetime()                   # extract last date in DataFrame
date_lowerbound = last_date_in_data + relativedelta(months=1)                  # next month
date_upperbound = last_date_in_data + relativedelta(years=YEARS_IN_FUTURE)     # future date to predict to

In [ ]:
pre_df = prediction_fx(ncrd_less, date_lowerbound.date(), date_upperbound.date())

In [ ]:
e_ci_df = prediction_exog_fx2(rd, exo_var_dict2, date_lowerbound.date(), date_upperbound.date())

In [ ]:
merged_ci_vals = pd.merge(pre_df, e_ci_df, left_index=True, right_index=True)

In [ ]:
merged_ci_vals.to_csv(os.path.join(destdir,'predictions.csv'))